<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=05734895c658d82a2d3a46a128aeb31f6e59f716ed6fdae5d4659c5408f1dfe6
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas-ta


In [2]:
# import libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()
import pandas_ta as ta

In [3]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-5*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [4]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')

mypf = mypf[mypf['InPortfolio'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()
stock_ids

Mounted at /content/drive


array(['5PAISA', 'AAVAS', 'AMARAJABAT', 'AMBUJACEM', 'ASIANPAINT',
       'BAJAJFINSV', 'BAJFINANCE', 'BANDHANBNK', 'BATAINDIA', 'BAYERCROP',
       'CLEAN', 'DMART', 'FINEORG', 'GICRE', 'GILLETTE', 'GLAND',
       'HDFCAMC', 'ICICIGI', 'INDIGOPNTS', 'INDUSINDBK', 'INFY', 'IRCTC',
       'JMFINANCIL', 'JUBLFOOD', 'LUXIND', 'NIACL', 'PAGEIND',
       'PIDILITIND', 'PNB', 'RELAXO', 'RELIANCE', 'ROSSARI', 'TEAMLEASE',
       'UJJIVANSFB', 'VAIBHAVGBL', 'VALIANTORG', 'WIPRO'], dtype=object)

In [5]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20 
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'
    
    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = yfin.Ticker(stock_symbol).history(period='5y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    stock_df['Stock'] = stock_symbol
    
    stock_df['Close'] = round(stock_df['Close'],0)     
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'
                   ], axis=1, inplace=True)   
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    #current_pe = get_current_pe(stock_id)
    stock_id = stock_id.upper() + '.NS'    
    tmp = stock_prec_dev(stock_id)
    tmp['Prev_Close'] = tmp['Close'].shift(1)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Shares'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]   
    tmp['Criteria'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Criteria'].values[0]   
    tmp['Strategy'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Strategy'].values[0]   
    #tmp['Curr_PE'] = current_pe
    df_prec_dev = df_prec_dev.append(tmp)

In [6]:
df_prec_dev.reset_index(drop=True, inplace=True)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

In [7]:
# portfolio stocks

tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df['Investment'] = tmp_df['Avg_Cost'] * tmp_df['Shares']
tmp_df['Current'] = tmp_df['Close'] * tmp_df['Shares']
tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
tmp_df['Estimated'] = tmp_df['Max'] * tmp_df['Shares']
tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
tmp_df['Estimated P/L%'] = round((tmp_df['Estimated'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
tmp_df['Juice Left%'] = tmp_df['Estimated P/L%'] - tmp_df['Current P/L%'] 

investment = round(sum(tmp_df['Avg_Cost']*tmp_df['Shares']),0)
current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
previous = round(sum(tmp_df['Prev_Close']*tmp_df['Shares']),0)

today_pnl_amount = current-previous
today_pnl_percentage = round((current-previous)*100/previous,2)

curr_pnl_amount = current-investment
curr_pnl_percentage = round((current-investment)*100/investment,2)

estimate = round(sum(tmp_df['Max']*tmp_df['Shares']),0)
est_pnl_amount = estimate-investment
est_pnl_percentage = round((estimate-investment)*100/investment,2)

tmp_df['Weightage%'] = round(tmp_df['Investment']*100/investment,2)
tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['Weightage%'],0)

print('Investment: ', investment)
print('Current: ', current)
print('Today PnL: ', today_pnl_amount)
print('Today PnL %: ', today_pnl_percentage)
print('Current PnL: ', curr_pnl_amount)
print('Current PnL %: ', curr_pnl_percentage)
print('Estimate: ', estimate)
print('Estimate PnL: ', est_pnl_amount)
print('Estimate PnL %: ', est_pnl_percentage)

qualified stocks: 37
Investment:  1768819.0
Current:  1812898.0
Today PnL:  -10678.0
Today PnL %:  -0.59
Current PnL:  44079.0
Current PnL %:  2.49
Estimate:  3248811.0
Estimate PnL:  1479992.0
Estimate PnL %:  83.67


In [8]:
tmp_df.sort_values(by = 'Current P/L%', ascending=False)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
12,5034.0,FINEORG.NS,7196.0,5061.0,4489.96,7.0,V40N,NaN,31429.72,35238.0,35427.0,50372.0,-0.53,12.12,60.27,48.15,1.78,22.0
3,456.0,AMBUJACEM.NS,588.0,452.0,408.85,120.0,TBD,ATH,49062.00,54720.0,54240.0,70560.0,0.88,11.53,43.82,32.29,2.77,32.0
6,7037.0,BAJFINANCE.NS,7901.0,7010.0,6365.23,16.0,V40,ATH,101843.68,112592.0,112160.0,126416.0,0.39,10.55,24.13,13.58,5.76,61.0
0,367.0,5PAISA.NS,553.0,377.0,333.09,152.0,V40N,ATH,50629.68,55784.0,57304.0,84056.0,-2.65,10.18,66.02,55.84,2.86,29.0
33,37.0,UJJIVANSFB.NS,57.0,37.0,33.60,900.0,V40N,ATH,30240.00,33300.0,33300.0,51300.0,0.00,10.12,69.64,59.52,1.71,17.0
17,1216.0,ICICIGI.NS,1620.0,1253.0,1106.17,46.0,V40,ATH,50883.82,55936.0,57638.0,74520.0,-2.95,9.93,46.45,36.52,2.88,29.0
22,73.0,JMFINANCIL.NS,120.0,72.0,67.30,450.0,2T3Y,2T3Y,30285.00,32850.0,32400.0,54000.0,1.39,8.47,78.31,69.84,1.71,14.0
35,573.0,VALIANTORG.NS,1851.0,580.0,530.15,96.0,2T3Y,2T3Y,50894.40,55008.0,55680.0,177696.0,-1.21,8.08,249.15,241.07,2.88,23.0
27,2615.0,PIDILITIND.NS,2902.0,2634.0,2442.09,20.0,V40,ATH,48841.80,52300.0,52680.0,58040.0,-0.72,7.08,18.83,11.75,2.76,20.0
13,183.0,GICRE.NS,358.0,185.0,171.85,175.0,2T3Y,2T3Y,30073.75,32025.0,32375.0,62650.0,-1.08,6.49,108.32,101.83,1.70,11.0


In [9]:
tmp_df.sort_values(by = 'Estimated P/L%', ascending=False)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
35,573.0,VALIANTORG.NS,1851.0,580.0,530.15,96.0,2T3Y,2T3Y,50894.40,55008.0,55680.0,177696.0,-1.21,8.08,249.15,241.07,2.88,23.0
15,960.0,GLAND.NS,4316.0,952.0,1354.78,37.0,2T3Y,2T3Y,50126.86,35520.0,35224.0,159692.0,0.84,-29.14,218.58,247.72,2.83,-82.0
24,1494.0,LUXIND.NS,4546.0,1499.0,1499.87,34.0,V40N,2T3Y,50995.58,50796.0,50966.0,154564.0,-0.33,-0.39,203.09,203.48,2.88,-1.0
34,306.0,VAIBHAVGBL.NS,966.0,308.0,320.64,96.0,2T3Y,2T3Y,30781.44,29376.0,29568.0,92736.0,-0.65,-4.57,201.27,205.84,1.74,-8.0
7,257.0,BANDHANBNK.NS,723.0,260.0,252.96,200.0,2T3Y,2T3Y,50592.00,51400.0,52000.0,144600.0,-1.15,1.60,185.82,184.22,2.86,5.0
25,118.0,NIACL.NS,317.0,120.0,116.60,260.0,TBD,ATH,30316.00,30680.0,31200.0,82420.0,-1.67,1.20,171.87,170.67,1.71,2.0
32,2314.0,TEAMLEASE.NS,5281.0,2218.0,2239.86,14.0,V40N,2T3Y,31358.04,32396.0,31052.0,73934.0,4.33,3.31,135.77,132.46,1.77,6.0
1,1354.0,AAVAS.NS,3196.0,1379.0,1405.00,15.0,2T3Y,2T3Y,21075.00,20310.0,20685.0,47940.0,-1.81,-3.63,127.47,131.10,1.19,-4.0
18,1417.0,INDIGOPNTS.NS,3112.0,1440.0,1408.25,83.0,V40N,2T3Y,116884.75,117611.0,119520.0,258296.0,-1.60,0.62,120.98,120.36,6.61,4.0
13,183.0,GICRE.NS,358.0,185.0,171.85,175.0,2T3Y,2T3Y,30073.75,32025.0,32375.0,62650.0,-1.08,6.49,108.32,101.83,1.70,11.0


In [10]:
tmp_df.sort_values(by = 'Juice Left%', ascending=True)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
27,2615.0,PIDILITIND.NS,2902.0,2634.0,2442.09,20.0,V40,ATH,48841.80,52300.0,52680.0,58040.0,-0.72,7.08,18.83,11.75,2.76,20.0
4,3181.0,ASIANPAINT.NS,3542.0,3191.0,3048.30,17.0,V40,ATH,51821.10,54077.0,54247.0,60214.0,-0.31,4.35,16.20,11.85,2.93,13.0
30,2482.0,RELIANCE.NS,2811.0,2500.0,2464.49,21.0,V40,ATH,51754.29,52122.0,52500.0,59031.0,-0.72,0.71,14.06,13.35,2.93,2.0
6,7037.0,BAJFINANCE.NS,7901.0,7010.0,6365.23,16.0,V40,ATH,101843.68,112592.0,112160.0,126416.0,0.39,10.55,24.13,13.58,5.76,61.0
5,1466.0,BAJAJFINSV.NS,1907.0,1467.0,1449.21,35.0,V40,NaN,50722.35,51310.0,51345.0,66745.0,-0.07,1.16,31.59,30.43,2.87,3.0
3,456.0,AMBUJACEM.NS,588.0,452.0,408.85,120.0,TBD,ATH,49062.00,54720.0,54240.0,70560.0,0.88,11.53,43.82,32.29,2.77,32.0
17,1216.0,ICICIGI.NS,1620.0,1253.0,1106.17,46.0,V40,ATH,50883.82,55936.0,57638.0,74520.0,-2.95,9.93,46.45,36.52,2.88,29.0
26,38363.0,PAGEIND.NS,53549.0,39004.0,39840.50,2.0,V40,ATH,79681.00,76726.0,78008.0,107098.0,-1.64,-3.71,34.41,38.12,4.50,-17.0
8,1575.0,BATAINDIA.NS,2173.0,1581.0,1501.51,67.0,V40,ATH,100601.17,105525.0,105927.0,145591.0,-0.38,4.89,44.72,39.83,5.69,28.0
9,4258.0,BAYERCROP.NS,6002.0,4344.0,4125.00,7.0,V40N,NaN,28875.00,29806.0,30408.0,42014.0,-1.98,3.22,45.50,42.28,1.63,5.0


In [11]:
tmp_df.sort_values(by = 'Weightage%', ascending=False)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
18,1417.0,INDIGOPNTS.NS,3112.0,1440.0,1408.25,83.0,V40N,2T3Y,116884.75,117611.0,119520.0,258296.0,-1.60,0.62,120.98,120.36,6.61,4.0
6,7037.0,BAJFINANCE.NS,7901.0,7010.0,6365.23,16.0,V40,ATH,101843.68,112592.0,112160.0,126416.0,0.39,10.55,24.13,13.58,5.76,61.0
8,1575.0,BATAINDIA.NS,2173.0,1581.0,1501.51,67.0,V40,ATH,100601.17,105525.0,105927.0,145591.0,-0.38,4.89,44.72,39.83,5.69,28.0
26,38363.0,PAGEIND.NS,53549.0,39004.0,39840.50,2.0,V40,ATH,79681.00,76726.0,78008.0,107098.0,-1.64,-3.71,34.41,38.12,4.50,-17.0
14,4440.0,GILLETTE.NS,7529.0,4457.0,4479.47,14.0,V40,ATH,62712.58,62160.0,62398.0,105406.0,-0.38,-0.88,68.08,68.96,3.55,-3.0
30,2482.0,RELIANCE.NS,2811.0,2500.0,2464.49,21.0,V40,ATH,51754.29,52122.0,52500.0,59031.0,-0.72,0.71,14.06,13.35,2.93,2.0
4,3181.0,ASIANPAINT.NS,3542.0,3191.0,3048.30,17.0,V40,ATH,51821.10,54077.0,54247.0,60214.0,-0.31,4.35,16.20,11.85,2.93,13.0
11,3632.0,DMART.NS,5324.0,3547.0,3702.36,14.0,SS,ATH,51833.04,50848.0,49658.0,74536.0,2.40,-1.90,43.80,45.70,2.93,-6.0
23,480.0,JUBLFOOD.NS,903.0,486.0,468.21,109.0,SS,ATH,51034.89,52320.0,52974.0,98427.0,-1.23,2.52,92.86,90.34,2.89,7.0
35,573.0,VALIANTORG.NS,1851.0,580.0,530.15,96.0,2T3Y,2T3Y,50894.40,55008.0,55680.0,177696.0,-1.21,8.08,249.15,241.07,2.88,23.0


In [12]:
tmp_df.to_csv('/content/drive/My Drive/data/stocks/myStocks-Portfolio-Analysis.csv', index=False)

In [13]:
tmp_df.sort_values(by = 'Risk Ind', ascending=True)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
15,960.0,GLAND.NS,4316.0,952.0,1354.78,37.0,2T3Y,2T3Y,50126.86,35520.0,35224.0,159692.0,0.84,-29.14,218.58,247.72,2.83,-82.0
26,38363.0,PAGEIND.NS,53549.0,39004.0,39840.50,2.0,V40,ATH,79681.00,76726.0,78008.0,107098.0,-1.64,-3.71,34.41,38.12,4.50,-17.0
34,306.0,VAIBHAVGBL.NS,966.0,308.0,320.64,96.0,2T3Y,2T3Y,30781.44,29376.0,29568.0,92736.0,-0.65,-4.57,201.27,205.84,1.74,-8.0
11,3632.0,DMART.NS,5324.0,3547.0,3702.36,14.0,SS,ATH,51833.04,50848.0,49658.0,74536.0,2.40,-1.90,43.80,45.70,2.93,-6.0
1,1354.0,AAVAS.NS,3196.0,1379.0,1405.00,15.0,2T3Y,2T3Y,21075.00,20310.0,20685.0,47940.0,-1.81,-3.63,127.47,131.10,1.19,-4.0
10,1377.0,CLEAN.NS,2648.0,1406.0,1417.55,15.0,NaN,NaN,21263.25,20655.0,21090.0,39720.0,-2.06,-2.86,86.80,89.66,1.20,-3.0
14,4440.0,GILLETTE.NS,7529.0,4457.0,4479.47,14.0,V40,ATH,62712.58,62160.0,62398.0,105406.0,-0.38,-0.88,68.08,68.96,3.55,-3.0
24,1494.0,LUXIND.NS,4546.0,1499.0,1499.87,34.0,V40N,2T3Y,50995.58,50796.0,50966.0,154564.0,-0.33,-0.39,203.09,203.48,2.88,-1.0
2,625.0,AMARAJABAT.NS,988.0,626.0,626.83,48.0,V200,ATH,30087.84,30000.0,30048.0,47424.0,-0.16,-0.29,57.62,57.91,1.70,-0.0
31,845.0,ROSSARI.NS,1569.0,849.0,845.85,24.0,NaN,ATH,20300.40,20280.0,20376.0,37656.0,-0.47,-0.10,85.49,85.59,1.15,-0.0
